In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
___ = ''

In [ ]:
pd.__version__

# Review over Pandas DataFrames

## Load the Data

"World Fertility Rates" from the StatsModels repository

In [ ]:
df = pd.read_csv('worldbankdata.csv')
df

## Selecting Data

Pandas has a lot of flexibility in the number of syntaxes it supports.  For example, to select columns in a DataFrame:

```python
df['Column1']
df.Column1
df.get('Column1')
```

Multiple Columns can also be selected by providing a list:

```python
df[['Column1', 'Column2']]
df.get(['Column1', 'Column2'])
```

Rows are selected with the **iloc** and **loc** attributes:

```python
df.iloc[5]  # Used to get the "integer" index of the row.
df.loc['Row5']  # Used if rows are named.
```

If you want to label the rows, you can make an index by using the **set_index()** method or by assinging the **DataFrame.index** property:

```python
df = df.set_index('Column1')
df.index = df['Column1']
```

You can switch the rows and columns using the **transpose()** method or **T** propoerty:

```python
df.transpose()
df.T
```


#### Exercises
Get the "Country Name" column from the data frame using the different methods:

In [ ]:
df[___]

In [ ]:
df.___

In [ ]:
df.get(___)

And the same with the year column **1976**:

In [ ]:
df[___]

In [ ]:
df.___

In [ ]:
df.get(___)

What about the 10th row of data?

In [ ]:
df.loc[]

In [ ]:
df.iloc[]

What about just Germany' data?

In [ ]:
df2 = df.set_index(___)
df2.___[___]

**Note:** When selecting single columns, Pandas will give a Series object instead of a DataFrame. If you'd like to keep it as a DataFrame, you can put the column name in a list:

```python
df[['Column1']]
df.get(['Column1'])
```

**Discussion**: What data selection syntax do you prefer?  What are some pros and cons of each?

## Filtering Data

To get rows based on their value, Pandas supports both Numpy's logical indexing for filtering rows and an SQL-like query string.  For example, to get all the rows of a dataframe that is positive for Column1:

```python
positive_rows = df['Column1'] > 0
df[positive_rows]
```

Often, this is done in a single line:

```python
df[df['Column1'] > 0]
```

And using the query() method:

```python
df.query('Column1 > 0')
```


#### Exercises
Try the three different filtering methods for each getting Germany's Data:

In [ ]:
germany_row = df[___] == ___
df[___]

In [ ]:
df[___]

In [ ]:
df.query(___)

Also, for the fertility rates above 10 in 2005:

In [ ]:
df[___]

In [ ]:
df.query(___)

**Discussion**: How might you approach this if you wanted just the fertility rates above 10, regardless of year or country, but still wanted to know which country and year the values appeared in?

## Transforming Data: Broadcasting Element-Wise Operations

Any transformation function can be performed on each element of a column, or even all columns of a DataFrame.  Here are several options for this approach:

Numpy-like Operator syntax:
```python
df['Column1'] * 100
```

Functions-style syntax:
```python
np.sqrt(df['Column1'])
```

Special broadcasted-version of different classes' Methods:
```python
df['Column1'].str.upper()
```

Apply() methods:
```python
df['Column1'].apply(np.sqrt)
```

****Exercises****: Let's try some of these on the data:

Square-root the values in 1970:

In [ ]:
np.___(df[___])

In [ ]:
df[___].apply(____)

Lowercase all of the country names using the string lower() method:

In [ ]:
df['Country Name'].___

In [ ]:
df['Country Name'].apply(___)

Replace all of the spaces with hyphens in Country Names (using the string replace() method):

In [ ]:
df['Country Name'].___

In [ ]:
df['Country Name'].apply(___)

## Summarizing / Aggregating Data in DataFrames

Pandas' Series and DataFrames are iterables, and can be given to any function that expects a list or Numpy Array, which allows them to be useful to many different libraries' functions.  For example, to compute a mean of a column:

```python
np.mean(df['Column1'])
```

You can also use the "agg" method to call a function on the rows or columns of a DataFrame:

```python
df['Column1'].agg(np.mean)
```

Finally, Pandas supplies many different aggregation functions as methods:

```python
df.mean()
```

#### Exercises

What is the standard deviation of the Fertility rates in 1970?

In [ ]:
df[___].___()

What is the difference between the maximum and minimum Fertility rate in the same year? (np.ptp)

In [ ]:
df[___].agg(___)

What the sum of squared diffferences of the data from its mean in the same year?

In [ ]:
def sum_squares(array):
    pass

df[___].agg(___)

What about the mean of Germany's fertility rates, across all years?

**Discussion**: What are some challenges you see with aggregations of country data in this dataset?

## Handling Missing Values

Missing values can be indicated with a None or np.nan value, like in Numpy.  Pandas Dataframes have several methods for detecting, removing, and replacing these values:

```python
df.isna()
df.notna()
df.dropna()
df.fillna()
df.interpolate()
```

In [ ]:
data = df.iloc[1][-10:]
data[4] = np.nan
data = data.astype(float)

In [ ]:
data

**Discussion**: What approaches to data imputation appear most often in your work?

## Rolling Windows

Rolling windows are used most often for smoothing data, particularly data that changes over time.  Pandas has a special "rolling()" method that can calculate aggregations over overlapping windows using a variety of methods:

In [ ]:
df['1968'].dropna().plot();

In [ ]:
df['1968'].dropna().rolling(20, center=True, win_type='hanning').mean().plot();

**Note**: The above example shows how methods can be "dot-chained" together to form mini-pipelines.  These can get quite long, so to make them a bit easier to read, sometimes you'll see them spread out on multiple lines using parentheses to glue them together:

In [ ]:
(df
 .get('1968')
 .dropna()
 .rolling(20, center=True, win_type='hanning')
 .mean()
 .plot()
);

## Long (aka "Tidy") DataFrames

This dataset is concise, but challenging to analyze, because it contains a mixed structure that mixes data labels and values in a variety of ways. Let's look at it again:

In [ ]:
df.head(10)

This mixed structure means that, depending on the columns used, we need to write different code to answer our questions.  We can simplify our code by reorganizing the data into a simpler table-like structure using the pandas **melt()** method:

In [ ]:
dfl = df.melt(
    id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], 
    var_name='Year', value_vars=df.columns[df.columns.str.isdigit()], 
    value_name='Fertility'
)
dfl.head()

Long tables are particularly nice, because they contain a lot more information about the dataset!

We can undo the operation, making a new "wide" table with the **pivot()** method:

In [ ]:
dfl.pivot(index='Year', columns='Country Name', values='Fertility').head()

In addition, you can explore different rotations of your data using the stack() and unstack() methods:

In [ ]:
dfl.unstack()

In [ ]:
dfl.stack()

## Aggregating Data by Groups:

Once your data is in a simple structure, there are many different ways that you can summarize your data, particularly when there are categorical factors in your dataset.  For example, you can pivot your data and aggregate along multiple axes:

In [ ]:
dfl.pivot_table(values='Fertility', index='Year', aggfunc='mean').head()

In [ ]:
dfl.pivot_table(values='Fertility', index='Year', columns='Country Name', aggfunc='mean').head()

You can create counts of your different combinations by building a contingincy table:

In [ ]:
pd.crosstab(dfl['Country Name'], dfl['Year'])

The most common approach, however, is the **groupby()** method, which works similarly to the rolling() method, in that you calculate an aggregate on a special object:

In [ ]:
dfl.groupby('Year').mean().head()

In [ ]:
dfl.groupby(['Country Name', 'Year']).mean().head()

You can also use groupby in a for-loop, in case it's desired:

In [ ]:
for country_name, dd in df.groupby('Country Name'):
    pass

**Note**: Because these approaches create new indices, you'll often see a **reset_index()** call afterwards to restore a simple columnar-only "table"-like structure.

## Joining DataFrames

Finally, if we have multiple datasets, we can join them together by matching up their rows via a "join" function or "merge" method call.  To explore this, let's get some more data: the Continent Name of each Country.

In [ ]:
url = 'http://techslides.com/list-of-countries-and-capitals'
dfc = pd.read_html(url, header=0)[0]
dfc

We'll use the the **merge()** function to match the two datasets on the "Country Name" columns:

In [ ]:
dfl2 = pd.merge(dfl, dfc, on='Country Name')
dfl2.head()

Now, we can group by continents!

In [ ]:
dfl2.groupby("Continent Name")['Fertility'].mean()

## Exercises 
Let's practice answering some more-complex questions with our new structure!

How could we *only* add the continent name to the dataset, without all the longitude/latitute info and duplicated codes?

How can we track the the variance of the continent's fertility rate between its countries, over time?

How many countries are in each continent?

How well is each continent's data tracked? (e.g. what percent of missing data is in each continent?)

What was the average fertility rate of the three "Americas" during the 1990s?

## Visualization

Long dataframes make for simpler visualization, particularly with the seaborn and plotly-express packages:

In [ ]:
import seaborn as sns
dfl2.Year = dfl2.Year.astype(int)
sns.lineplot(data=dfl2, x='Year', y='Fertility', hue='Continent Name', );

## Writing Data

Pandas supports a wealth of data formats, all with a **DataFrame.to_()** method.  Let's take a look at some of these and discuss some gotchas we've come across when using them.

In [ ]:
dfl2.to_csv("continent_fertility.csv")